In [69]:
from collections import Counter

a = Counter()
articles = []

strings = "stringt"
c = 100

def stock(text):
    for alpha in text:
        a[alpha]+=1  

a

Counter()

In [70]:
stock(strings)
a.most_common()

[('t', 2), ('s', 1), ('r', 1), ('i', 1), ('n', 1), ('g', 1)]

In [71]:
strings = "geeksforgeeks"
stock(strings)
a.most_common()

[('e', 4),
 ('s', 3),
 ('g', 3),
 ('t', 2),
 ('r', 2),
 ('k', 2),
 ('i', 1),
 ('n', 1),
 ('f', 1),
 ('o', 1)]

### SCRAPING TWEETS

### TRAINING ANAGO MODEL

In [1]:
#  mempersiapkan environment anago
import anago
from anago.reader import load_data_and_labels
import os
import numpy as np
import random as rn

Using TensorFlow backend.
D:\ITS\Mata Kuliah\Tugas Akhir\TAFinal\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ITS\Mata Kuliah\Tugas Akhir\TAFinal\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ITS\Mata Kuliah\Tugas Akhir\TAFinal\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ITS\Mata Kuliah\Tuga

In [2]:
AnagoModel = anago.Sequence().load("AnagoModel")

In [64]:
namaFileTrain = "train.txt"
namaFileValid = "valid.txt"
namaFileTest = "test.txt"
x_train, y_train = load_data_and_labels(namaFileTrain)
x_valid, y_valid = load_data_and_labels(namaFileValid)
x_test, y_test = load_data_and_labels(namaFileTest)

In [67]:
print("\n\nEvaluasi Test:")
AnagoModel.eval(x_test, y_test)

words = 'Budi Martami kuliah di UPI yang berlokasi di Bandung'.split()
print(AnagoModel.analyze(words))

words = 'PDIP yang dikawal Megawati menang dalam Pilkada DKI Jakarta'.split()
print(AnagoModel.analyze(words))

print("\n\nEvaluasi Test:")
AnagoModel.eval(x_test, y_test)

words = 'Megawati adalah ketua PDIP yang merupakan orang Bandung'.split()
print(AnagoModel.analyze(words))

words = 'Proklamasi kemerdekaan Indonesia berlangsung di Jakarta'.split()
print(AnagoModel.analyze(words))



Evaluasi Test:
 - f1: 92.35
{'words': ['Budi', 'Martami', 'kuliah', 'di', 'UPI', 'yang', 'berlokasi', 'di', 'Bandung'], 'entities': [{'text': 'Budi Martami', 'type': 'OBJ', 'score': 1.0, 'beginOffset': 0, 'endOffset': 2}, {'text': 'Bandung', 'type': 'GPE', 'score': 1.0, 'beginOffset': 8, 'endOffset': 9}]}
{'words': ['PDIP', 'yang', 'dikawal', 'Megawati', 'menang', 'dalam', 'Pilkada', 'DKI', 'Jakarta'], 'entities': [{'text': 'DKI Jakarta', 'type': 'GPE', 'score': 1.0, 'beginOffset': 7, 'endOffset': 9}]}


Evaluasi Test:
 - f1: 92.35
{'words': ['Megawati', 'adalah', 'ketua', 'PDIP', 'yang', 'merupakan', 'orang', 'Bandung'], 'entities': [{'text': 'ketua', 'type': 'OBJ', 'score': 1.0, 'beginOffset': 2, 'endOffset': 3}, {'text': 'orang Bandung', 'type': 'OBJ', 'score': 1.0, 'beginOffset': 6, 'endOffset': 8}]}
{'words': ['Proklamasi', 'kemerdekaan', 'Indonesia', 'berlangsung', 'di', 'Jakarta'], 'entities': [{'text': 'Indonesia', 'type': 'GPE', 'score': 1.0, 'beginOffset': 2, 'endOffset': 3

### TWEETS FILTRATION USING NER

In [3]:
## loading data
import pandas as pd
tweets = pd.read_csv('Tweets/gerhanaMatahariCincin.csv')
#tweets.head(5)

In [4]:
## preprocessing tweets (#1)
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import html
import re

# nlp = Indonesian()
# pd.set_option('max_colwidth', 100)
# stopwords = spacy.lang.id.stop_words.STOP_WORDS

# load sastrawi's Bahasa Indonesia stopwords as variable called 'stopwords'
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
extender = ["ada", "adalah", "adanya", "adapun", "agak", "agaknya", "agar", "akan", "akankah", "akhir", "akhiri", "akhirnya", "aku", "akulah", "amat", "amatlah", "anda", "andalah", "antar", "antara", "antaranya", "apa", "apaan", "apabila", "apakah", "apalagi", "apatah", "artinya", "asal", "asalkan", "atas", "atau", "ataukah", "ataupun", "awal", "awalnya", "bagai", "bagaikan", "bagaimana", "bagaimanakah", "bagaimanapun", "bagi", "bagian", "bahkan", "bahwa", "bahwasanya", "baik", "bakal", "bakalan", "balik", "banyak", "bapak", "baru", "bawah", "beberapa", "begini", "beginian", "beginikah", "beginilah", "begitu", "begitukah", "begitulah", "begitupun", "bekerja", "belakang", "belakangan", "belum", "belumlah", "benar", "benarkah", "benarlah", "berada", "berakhir", "berakhirlah", "berakhirnya", "berapa", "berapakah", "berapalah", "berapapun", "berarti", "berawal", "berbagai", "berdatangan", "beri", "berikan", "berikut", "berikutnya", "berjumlah", "berkali-kali", "berkata", "berkehendak", "berkeinginan", "berkenaan", "berlainan", "berlalu", "berlangsung", "berlebihan", "bermacam", "bermacam-macam", "bermaksud", "bermula", "bersama", "bersama-sama", "bersiap", "bersiap-siap", "bertanya", "bertanya-tanya", "berturut", "berturut-turut", "bertutur", "berujar", "berupa", "besar", "betul", "betulkah", "biasa", "biasanya", "bila", "bilakah", "bisa", "bisakah", "boleh", "bolehkah", "bolehlah", "buat", "bukan", "bukankah", "bukanlah", "bukannya", "bulan", "bung", "cara", "caranya", "cukup", "cukupkah", "cukuplah", "cuma", "dahulu", "dalam", "dan", "dapat", "dari", "daripada", "datang", "dekat", "demi", "demikian", "demikianlah", "dengan", "depan", "di", "dia", "diakhiri", "diakhirinya", "dialah", "diantara", "diantaranya", "diberi", "diberikan", "diberikannya", "dibuat", "dibuatnya", "didapat", "didatangkan", "digunakan", "diibaratkan", "diibaratkannya", "diingat", "diingatkan", "diinginkan", "dijawab", "dijelaskan", "dijelaskannya", "dikarenakan", "dikatakan", "dikatakannya", "dikerjakan", "diketahui", "diketahuinya", "dikira", "dilakukan", "dilalui", "dilihat", "dimaksud", "dimaksudkan", "dimaksudkannya", "dimaksudnya", "diminta", "dimintai", "dimisalkan", "dimulai", "dimulailah", "dimulainya", "dimungkinkan", "dini", "dipastikan", "diperbuat", "diperbuatnya", "dipergunakan", "diperkirakan", "diperlihatkan", "diperlukan", "diperlukannya", "dipersoalkan", "dipertanyakan", "dipunyai", "diri", "dirinya", "disampaikan", "disebut", "disebutkan", "disebutkannya", "disini", "disinilah", "ditambahkan", "ditandaskan", "ditanya", "ditanyai", "ditanyakan", "ditegaskan", "ditujukan", "ditunjuk", "ditunjuki", "ditunjukkan", "ditunjukkannya", "ditunjuknya", "dituturkan", "dituturkannya", "diucapkan", "diucapkannya", "diungkapkan", "dong", "dua", "dulu", "empat", "enggak", "enggaknya", "entah", "entahlah", "guna", "gunakan", "hal", "hampir", "hanya", "hanyalah", "hari", "harus", "haruslah", "harusnya", "hendak", "hendaklah", "hendaknya", "hingga", "ia", "ialah", "ibarat", "ibaratkan", "ibaratnya", "ibu", "ikut", "ingat", "ingat-ingat", "ingin", "inginkah", "inginkan", "ini", "inikah", "inilah", "itu", "itukah", "itulah", "jadi", "jadilah", "jadinya", "jangan", "jangankan", "janganlah", "jauh", "jawab", "jawaban", "jawabnya", "jelas", "jelaskan", "jelaslah", "jelasnya", "jika", "jikalau", "juga", "jumlah", "jumlahnya", "justru", "kala", "kalau", "kalaulah", "kalaupun", "kalian", "kami", "kamilah", "kamu", "kamulah", "kan", "kapan", "kapankah", "kapanpun", "karena", "karenanya", "kasus", "kata", "katakan", "katakanlah", "katanya", "ke", "keadaan", "kebetulan", "kecil", "kedua", "keduanya", "keinginan", "kelamaan", "kelihatan", "kelihatannya", "kelima", "keluar", "kembali", "kemudian", "kemungkinan", "kemungkinannya", "kenapa", "kepada", "kepadanya", "kesampaian", "keseluruhan", "keseluruhannya", "keterlaluan", "ketika", "khususnya", "kini", "kinilah", "kira", "kira-kira", "kiranya", "kita", "kitalah", "kok", "kurang", "lagi", "lagian", "lah", "lain", "lainnya", "lalu", "lama", "lamanya", "lanjut", "lanjutnya", "lebih", "lewat", "lima", "luar", "macam", "maka", "makanya", "makin", "malah", "malahan", "mampu", "mampukah", "mana", "manakala", "manalagi", "masa", "masalah", "masalahnya", "masih", "masihkah", "masing", "masing-masing", "mau", "maupun", "melainkan", "melakukan", "melalui", "melihat", "melihatnya", "memang", "memastikan", "memberi", "memberikan", "membuat", "memerlukan", "memihak", "meminta", "memintakan", "memisalkan", "memperbuat", "mempergunakan", "memperkirakan", "memperlihatkan", "mempersiapkan", "mempersoalkan", "mempertanyakan", "mempunyai", "memulai", "memungkinkan", "menaiki", "menambahkan", "menandaskan", "menanti", "menanti-nanti", "menantikan", "menanya", "menanyai", "menanyakan", "mendapat", "mendapatkan", "mendatang", "mendatangi", "mendatangkan", "menegaskan", "mengakhiri", "mengapa", "mengatakan", "mengatakannya", "mengenai", "mengerjakan", "mengetahui", "menggunakan", "menghendaki", "mengibaratkan", "mengibaratkannya", "mengingat", "mengingatkan", "menginginkan", "mengira", "mengucapkan", "mengucapkannya", "mengungkapkan", "menjadi", "menjawab", "menjelaskan", "menuju", "menunjuk", "menunjuki", "menunjukkan", "menunjuknya", "menurut", "menuturkan", "menyampaikan", "menyangkut", "menyatakan", "menyebutkan", "menyeluruh", "menyiapkan", "merasa", "mereka", "merekalah", "merupakan", "meski", "meskipun", "meyakini", "meyakinkan", "minta", "mirip", "misal", "misalkan", "misalnya", "mula", "mulai", "mulailah", "mulanya", "mungkin", "mungkinkah", "nah", "naik", "namun", "nanti", "nantinya", "nyaris", "nyatanya", "oleh", "olehnya", "pada", "padahal", "padanya", "pak", "paling", "panjang", "pantas", "para", "pasti", "pastilah", "penting", "pentingnya", "per", "percuma", "perlu", "perlukah", "perlunya", "pernah", "persoalan", "pertama", "pertama-tama", "pertanyaan", "pertanyakan", "pihak", "pihaknya", "pukul", "pula", "pun", "punya", "rasa", "rasanya", "rata", "rupanya", "saat", "saatnya", "saja", "sajalah", "saling", "sama", "sama-sama", "sambil", "sampai", "sampai-sampai", "sampaikan", "sana", "sangat", "sangatlah", "satu", "saya", "sayalah", "se", "sebab", "sebabnya", "sebagai", "sebagaimana", "sebagainya", "sebagian", "sebaik", "sebaik-baiknya", "sebaiknya", "sebaliknya", "sebanyak", "sebegini", "sebegitu", "sebelum", "sebelumnya", "sebenarnya", "seberapa", "sebesar", "sebetulnya", "sebisanya", "sebuah", "sebut", "sebutlah", "sebutnya", "secara", "secukupnya", "sedang", "sedangkan", "sedemikian", "sedikit", "sedikitnya", "seenaknya", "segala", "segalanya", "segera", "seharusnya", "sehingga", "seingat", "sejak", "sejauh", "sejenak", "sejumlah", "sekadar", "sekadarnya", "sekali", "sekali-kali", "sekalian", "sekaligus", "sekalipun", "sekarang", "sekarang", "sekecil", "seketika", "sekiranya", "sekitar", "sekitarnya", "sekurang-kurangnya", "sekurangnya", "sela", "selain", "selaku", "selalu", "selama", "selama-lamanya", "selamanya", "selanjutnya", "seluruh", "seluruhnya", "semacam", "semakin", "semampu", "semampunya", "semasa", "semasih", "semata", "semata-mata", "semaunya", "sementara", "semisal", "semisalnya", "sempat", "semua", "semuanya", "semula", "sendiri", "sendirian", "sendirinya", "seolah", "seolah-olah", "seorang", "sepanjang", "sepantasnya", "sepantasnyalah", "seperlunya", "seperti", "sepertinya", "sepihak", "sering", "seringnya", "serta", "serupa", "sesaat", "sesama", "sesampai", "sesegera", "sesekali", "seseorang", "sesuatu", "sesuatunya", "sesudah", "sesudahnya", "setelah", "setempat", "setengah", "seterusnya", "setiap", "setiba", "setibanya", "setidak-tidaknya", "setidaknya", "setinggi", "seusai", "sewaktu", "siap", "siapa", "siapakah", "siapapun", "sini", "sinilah", "soal", "soalnya", "suatu", "sudah", "sudahkah", "sudahlah", "supaya", "tadi", "tadinya", "tahu", "tahun", "tak", "tambah", "tambahnya", "tampak", "tampaknya", "tandas", "tandasnya", "tanpa", "tanya", "tanyakan", "tanyanya", "tapi", "tegas", "tegasnya", "telah", "tempat", "tengah", "tentang", "tentu", "tentulah", "tentunya", "tepat", "terakhir", "terasa", "terbanyak", "terdahulu", "terdapat", "terdiri", "terhadap", "terhadapnya", "teringat", "teringat-ingat", "terjadi", "terjadilah", "terjadinya", "terkira", "terlalu", "terlebih", "terlihat", "termasuk", "ternyata", "tersampaikan", "tersebut", "tersebutlah", "tertentu", "tertuju", "terus", "terutama", "tetap", "tetapi", "tiap", "tiba", "tiba-tiba", "tidak", "tidakkah", "tidaklah", "tiga", "tinggi", "toh", "tunjuk", "turut", "tutur", "tuturnya", "ucap", "ucapnya", "ujar", "ujarnya", "umum", "umumnya", "ungkap", "ungkapnya", "untuk", "usah", "usai", "waduh", "wah", "wahai", "waktu", "waktunya", "walau", "walaupun", "wong", "yaitu", "yakin", "yakni", "yang"]
stopwords.extend(extender)
# print(stopwords)
def normalize_repeated_char(sentence):
    alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
                'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
                'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    for i in range(len(alphabet)):
        charac_long = 5
        while charac_long > 2:
            char = alphabet[i] * charac_long
            sentence = sentence.replace(char, alphabet[i])
            charac_long -= 1
    return sentence

def delete_emoji(sentence):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    for word in sentence:
        for letter in word:
            print(letter)
#             letter = emoji_pattern.sub(r'', letter)
#     deleted_emoji = emoji_pattern.sub(r'', ContentNoSlang)
#     return deleted_emoji

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')


def remove_stopword(words):
    clean = []
    for word in words:
        if word not in stopwords:
            clean.append(word)
    return (" ".join(clean))

def preprocessing_tweets_first(text):
    clean = html.unescape(text) #Handling Unicode HTML
    clean = normalize_repeated_char(clean) 
    clean = remove_stopword(clean.split())
    clean = re.sub("[0-9]", "", clean) #Remove Number
    clean = re.sub(r"#\S+", "", clean) #Remove hashtag
    clean = re.sub(r"http\S+", "", clean) #Remove links
    clean = re.sub(r"pic.twitter\S+", "", clean) #Remove image links
    clean = re.sub(r"null", "", clean) #Remove null
    clean = deEmojify(clean) #Remove Emoji
    clean = " ".join(re.findall("[a-zA-Z]{3,}", clean)) #Remove Punctuation
    content = []
    content.append(clean)
    return " ".join(content)

In [6]:
# executing the preprocessing
tweets['clean'] = tweets['tweet'].apply(preprocessing_tweets_first)
tweets = tweets.drop(["id","timezone","user_id","name","place","mentions","urls","photos","replies_count","retweets_count","likes_count","hashtags","cashtags","link","retweet","quote_url","video","near","geo","source","user_rt_id","user_rt","retweet_id","reply_to","retweet_date","translate","trans_src","trans_dest","id","conversation_id","date","time","username","created_at"], axis=1)

In [9]:
tweets['tweet'][5]

'Tanam anak hidup-hidup ketika gerhana matahari cincin\n https://www.bazookapenaka.com.my/?p=60784\xa0'

In [10]:
tweets.head(10)

,tweet,clean
0,oke aziiiiii cewenya anak senja senja mendung ...,oke azii cewenya anak senja senja mendung gerh...
1,Siak dan Singkawang terbaik untuk melihat gerh...,Siak Singkawang terbaik gerhana matahari cinci...
2,Ini Rangkaian Kegiatan Festival Gerhana Mataha...,Ini Rangkaian Kegiatan Festival Gerhana Mataha...
3,"Setelah Natal, Gerhana Matahari Cincin Melinta...",Setelah Natal Gerhana Matahari Cincin Melintas...
4,Tanam anak ketika gerhana matahari \n\nNEW DEL...,Tanam anak gerhana matahari NEW DELHI Sekumpul...
5,Tanam anak hidup-hidup ketika gerhana matahari...,Tanam anak hidup hidup gerhana matahari cincin
6,Masih dikasih kesempatan sm Allah utk shalat g...,Masih dikasih kesempatan Allah utk shalat gerh...
7,"Kasihan matahari. Terlalu terik, dikutuk merek...",Kasihan matahari Terlalu terik dikutuk takut h...
8,Kabupaten Siak menjadi salah satu daerah yang ...,Kabupaten Siak salah daerah berkesempatan meny...
9,Kemaren ada gerhana matahari cincin hari ini a...,Kemaren gerhana matahari cincin cincin jari Yi...


In [33]:
## filtering tweets with entity
results = []
tweets_with_entity = pd.DataFrame(columns=['tweet', 'clean'])
for index, row in tweets.iterrows():
#     index+=1
    if len(row['clean'].split()) >= 3: # filtering tweets that contain < 3 words
        if row['clean'] not in tweets_with_entity['clean'].tolist(): # filtering exactly same tweet
            result = AnagoModel.analyze(row['clean'].split())

            if result.get("entities") != []:
                tweets_with_entity = tweets_with_entity.append({'tweet':row['tweet'], 'clean':row['clean']}, ignore_index=True)
                results.append(result)

In [9]:
results

[{'words': ['News',
   'Polres',
   'Jaksel',
   'Terus',
   'Telusuri',
   'Indikasi',
   'Peralihan',
   'Kepemilikan',
   'Lamborghini',
   'Sang',
   'Koboi'],
  'entities': [{'text': 'News Polres',
    'type': 'OBJ',
    'score': 1.0,
    'beginOffset': 0,
    'endOffset': 2}]},
 {'words': ['Polres',
   'Jaksel',
   'Terus',
   'Telusuri',
   'Indikasi',
   'Peralihan',
   'Kepemilikan',
   'Lamborghini',
   'Sang',
   'Koboi'],
  'entities': [{'text': 'Polres',
    'type': 'OBJ',
    'score': 1.0,
    'beginOffset': 0,
    'endOffset': 1}]},
 {'words': ['Polres',
   'Jaksel',
   'Terus',
   'Menelusuri',
   'Indikasi',
   'Peralihan',
   'Kepemilikan',
   'Lamborghini',
   'Sang',
   'Koboi',
   'News',
   'TauCepatTanpaBatas',
   'BeritaTerkini',
   'BeritaTerkini',
   'NewsUpdate'],
  'entities': [{'text': 'Polres',
    'type': 'OBJ',
    'score': 1.0,
    'beginOffset': 0,
    'endOffset': 1}]},
 {'words': ['Polres',
   'Jaksel',
   'Terus',
   'Menelusuri',
   'Indikasi',
   

### TRENDING ISSUE EXTRACTION

In [37]:
from pyclustering.cluster.kmedoids import kmedoids
from pyclustering.cluster import cluster_visualizer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES

#### Clustering Tweets

###### preprocessing (#2)

In [38]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# https://medium.com/@arie.pratama.s/bahasa-indonesia-open-sourced-nlp-resources-8cb394193238
def normalize_slang_words(text):
    slang_word_dict = json.loads(open('slang_word_dict.txt', 'r').read())
    words = text.split()
    for index in range(len(words)):
        for key, value in slang_word_dict.items():
            for v in value:
                if words[index] == v:
                    words[index] = key
                else:
                    continue 
    content = list()
    content.append(words)
    return " ".join(content)

def preprocessing_tweets_second(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    clean   = stemmer.stem(text) 
    clean = re.sub("#", "", clean)
    clean
#     clean = normalize_slang_words(clean)
    return clean

# for index, row in tweets.iterrows():
# #     index+=1
#     if len(row['clean'].split()) >= 3:
#         result = AnagoModel.analyze(row['clean'].split())
#         if result.get("entities") != []:
#             tweets_with_entity = tweets_with_entity.append({'text':row['tweet'], 'clean':row['clean']}, ignore_index=True)
#             results.append(result)

In [39]:
tweets_with_entity['clean2'] = tweets_with_entity['clean'].apply(preprocessing_tweets_second)

tweets_with_entity

,tweet,clean,clean2
0,#News: Polres Jaksel Terus Telusuri Indikasi P...,Polres Jaksel Terus Telusuri Indikasi Peraliha...,polres jaksel terus telusur indikasi alih mili...
1,Polres Jaksel Terus Menelusuri Indikasi Perali...,Polres Jaksel Terus Menelusuri Indikasi Perali...,polres jaksel terus telusur indikasi alih mili...
2,Polres Jaksel Terus Menelusuri Indikasi Perali...,Polres Jaksel Terus Menelusuri Indikasi Perali...,polres jaksel terus telusur indikasi alih mili...
3,TERKUAK! Deretan Kasus yang Bayangi 'Koboi' La...,TERKUAK Deretan Kasus Bayangi Koboi Lamborghin...,kuak deret kasus bayang koboi lamborghini beri...
4,"Kasus koboi lamborghini dblowup media TV, akhi...",Kasus koboi lamborghini dblowup media jiwasray...,kasus koboi lamborghini dblowup media jiwasray...
...,...,...,...
59,"Bak Koboi Jalanan, Pengemudi Lamborghini Todon...",Bak Koboi Jalanan Pengemudi Lamborghini Todong...,bak koboi jalan kemudi lamborghini todong ajar...
60,"Bak Koboi Jalanan, Pengemudi Lamborghini Todon...",Bak Koboi Jalanan Pengemudi Lamborghini Todong...,bak koboi jalan kemudi lamborghini todong ajar...
61,"@null Bak Koboi Jalanan, Pengemudi Lamborghini...",Bak Koboi Jalanan Pengemudi Lamborghini Todong...,bak koboi jalan kemudi lamborghini todong ajar...
62,Haha iya padahal baru 1 gelas mek RT @henrymhr...,Haha iya gelas mek henrymhrdk Dhan lau ajak ko...,haha iya gelas mek henrymhrdk dhan lau ajak ko...


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

# Vectorise the data
vec = TfidfVectorizer()
X = vec.fit_transform(tweets_with_entity['clean2'].tolist()) # `X` will now be a TF-IDF representation of the data, the first row of `X` corresponds to the first sentence in `data`

# Calculate the pairwise cosine similarities (depending on the amount of data that you are going to have this could take a while)
S = cosine_similarity(X)

In [27]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
#     print(filtered_tokens)
    return filtered_tokens    

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
import pandas as pd
import nltk

# creating bag of words
words = []
for sent in tweets_with_entity['clean2'].tolist():
    toks = tokenize_only(sent)
    for tok in toks:
        if tok not in words:
            words.append(tok)
        
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(vocabulary=words, max_df=0.8, max_features=200000,
                                 min_df = 0.2, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_with_entity['clean2'].tolist()) #fit the vectorizer to synopses

Wall time: 83.8 ms


In [44]:
# Clustering menggunakan K-Means dengan fitur tf-idf
## jangan di apa-apain dulu, memperlihatkan ekstraksi isu yang nyaris berhasil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# documents = indonesia['clean'].tolist()
# vectorizer = TfidfVectorizer(stop_words=stopwords)
# X = vectorizer.fit_transform(documents)

true_k = 5
kMeans = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
kMeans.fit(tfidf_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=5, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [45]:
## extracting cluster
# https://github.com/akanshajainn/K-means-Clustering-on-Text-Documents/blob/master/doc_clustering.py
corpus = tweets_with_entity['clean2'].tolist()
clusters = kMeans.labels_.tolist()
idea={'Idea':corpus, 'Cluster':clusters} #Creating dict having doc with the corresponding cluster number.
frame=pd.DataFrame(idea,index=[clusters], columns=['Idea','Cluster']) # Converting it into a dataframe.


## Trending Issues

In [49]:
print(frame['Cluster'].value_counts()) #Print the counts of doc belonging to each cluster.

2    27
1    16
3    10
0     7
4     4
Name: Cluster, dtype: int64


In [50]:
# creating list of cluster
c = 0
clusters = []
while c < true_k:
    clusters.append((frame.loc[frame['Cluster'] == c])['Idea'].tolist())
    c+=1

#### Selecting Cluster (Contain most tweets)

In [52]:
print(frame['Cluster'].value_counts().sort_index()) #Print the counts of doc belonging to each cluster.
cluster_with_most_tweet = (frame['Cluster'].value_counts().sort_values()).index.tolist()[-1]
print("\nCluster containing most tweet: "+ str(cluster_with_most_tweet))

0     7
1    16
2    27
3    10
4     4
Name: Cluster, dtype: int64

Cluster containing most tweet: 2


#### Selecting Cluster (Cluster Importance)

In [65]:
import math 
cluster_score = []
for cluster in clusters:
    # creating tf per cluster
    TF = pd.DataFrame(columns=['term','frequency'])
    for sentence in cluster:
        for word in sentence.split():
            if word not in TF['term'].tolist():
                TF = TF.append(pd.DataFrame({"term":[word], "frequency":[1]}), ignore_index = True)
            else:
                TF.loc[TF.term == word, 'frequency'] = int(TF.loc[TF.term == word, 'frequency'])+1
    # counting CI per cluster
    cluster_importance = 0
    for index, row in TF.iterrows():
        cluster_importance += math.log(1+int(row['frequency']))
    cluster_score.append(cluster_importance)
    
print("Cluster_socre:")
print(cluster_score)

best_cluster = cluster_score.index(max(cluster_score))
print("\nCluster with highest cluster importance: " + str(best_cluster))

Cluster_socre:
[26.787008477286133, 84.03400859928637, 163.8660121281035, 64.5253596227882, 22.744779916203036]

Cluster with highest cluster importance: 2


#### extracting issue

#### Most representative words

In [96]:
## Extracting Most representative words for each cluster
print("Top terms per cluster:")
order_centroids = kMeans.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
clusters_top_terms = []
for i in range(true_k):
    cluster_top_terms = []
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
        cluster_top_terms.append(terms[ind])
    clusters_top_terms.append(cluster_top_terms)
print(clusters_top_terms)

Top terms per cluster:
Cluster 0:
 harga
 jalan
 sang
 mahal
 nyata
 lamborghini
 serabutan
 koboi
 buruh
 mobil
Cluster 1:
 milik
 mobil
 palsu
 koboi
 lamborghini
 aksi
 pelat
 indikasi
 alih
 polres
Cluster 2:
 ajar
 todong
 kemudi
 lamborghini
 koboi
 kemang
 bak
 jalan
 aksi
 senjata
Cluster 3:
 sopir
 korban
 register
 orang
 nama
 buruh
 tangkap
 lamborghini
 koboi
 anak
Cluster 4:
 harimau
 sumatera
 simpan
 gerebek
 rumah
 opsetan
 awet
 sita
 denda
 penjara
[['harga', 'jalan', 'sang', 'mahal', 'nyata', 'lamborghini', 'serabutan', 'koboi', 'buruh', 'mobil'], ['milik', 'mobil', 'palsu', 'koboi', 'lamborghini', 'aksi', 'pelat', 'indikasi', 'alih', 'polres'], ['ajar', 'todong', 'kemudi', 'lamborghini', 'koboi', 'kemang', 'bak', 'jalan', 'aksi', 'senjata'], ['sopir', 'korban', 'register', 'orang', 'nama', 'buruh', 'tangkap', 'lamborghini', 'koboi', 'anak'], ['harimau', 'sumatera', 'simpan', 'gerebek', 'rumah', 'opsetan', 'awet', 'sita', 'denda', 'penjara']]


In [97]:
best_cluster_top_terms = clusters_top_terms[best_cluster]
best_cluster_top_terms

['ajar',
 'todong',
 'kemudi',
 'lamborghini',
 'koboi',
 'kemang',
 'bak',
 'jalan',
 'aksi',
 'senjata']

### NEWS SUMMARIZATION

In [13]:
## reading news XML
import xml.etree.ElementTree as et 
import os
directory = "./ScrapeNews/manually/Lamborghini/"
news_df = pd.DataFrame(columns=['id','judul','tanggal','tag','isi'])

for berita in os.listdir(directory):
    filename = directory+berita
    xtree = et.parse(filename).getroot()
    judul = xtree.find('judul').text
    ids = xtree.find('id').text
    tanggal = xtree.find('tanggal').text
    tag = xtree.find('tag').text
    isi = xtree.find('isi').text
    news_df = news_df.append({'id':ids,'judul':judul,'tanggal':tanggal,'tag':tag,'isi':isi},ignore_index=True)
#     news_df2 = pd.DataFrame([id,judul,tanggal,kata_kunci,isi], columns=['judul','id','tanggal','kata_kunci','isi'])
#     news_df = news_df.append(news_df2)

pd.set_option('max_colwidth', 30)
# news_df

In [14]:
news_df

,id,judul,tanggal,tag,isi
0,Lamborghini_CNN,Pengemudi Lamborghini Todo...,"CNN Indonesia | Selasa, 24...","lamborghini, aksi koboi pe...","Jakarta, CNN Indonesia -- ..."
1,Lamborghini_Detik,"Bak Koboi Jalanan, Pengemu...","Senin, 23 Des 2019 21:30 WIB","aksi koboi,aksi koboi peng...",Jakarta - Aksi arogan dila...
2,Lamborghini_Kompas,Geger Aksi Koboi Pengemudi...,"Kamis, 26 Desember 2019 | ...","Lamborghini koboi, Lamborg...",Kasus penodongan 2 pelajar...
3,Lamborghini_Kompas2,Polisi Tahan Pengembudi La...,"Rabu, 25 Desember 2019 | 0...","Mobil lamborghini, Penodon...",Polda Metro Jaya telah men...
4,Lamborghini_Merdeka,"Tembakkan Pistol ke Udara,...","Senin, 23 Desember 2019 23:06","Kriminal,Lamborghini,Penod...",Aksi koboi jalanan kembali...
5,Lamborghini_Tribun,Geger Aksi Koboi Pengemudi...,"Kamis, 26 Desember 2019 23...",None,Kasus penodongan 2pelajar ...


##### Preprocessing berita

In [29]:
# mamang
import nltk
import html
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


In [19]:
# factory = StopWordRemoverFactory()
# stopwords = factory.get_stop_words()
# stopwords.extend(extender)
def preprocessing_news(text):
    clean = html.unescape(text) #Handling Unicode HTML
    clean = re.sub("[0-9]", "", clean) #RemoveNumber
    clean = " ".join(re.findall("[#a-zA-Z]{3,}", clean)) #Remove Puntc
    clean = clean.lower()
    content = []
    content.append(clean)
    return " ".join(content)

# def preprocessing_news(text):
#      clean = html.unescape(text) #Handling Unicode HTML
#     clean = " ".join(re.findall("[#a-zA-Z]{3,}", clean)) #Remove Puntc
#     clean = text.lower() #lowercase # permrosesan NER sangat dipengaruhi huruf kapital
#     clean = stopword(clean) #Stopword
    return clean
#     content = list()
#     content.append(clean)
#     return " ".join(content)

news_df['clean'] = news_df['isi'].apply(preprocessing_news)
news_df.head(10)

,id,judul,tanggal,tag,isi,clean
0,Lamborghini_CNN,Pengemudi Lamborghini Todo...,"CNN Indonesia | Selasa, 24...","lamborghini, aksi koboi pe...","Jakarta, CNN Indonesia -- ...",jakarta cnn indonesia pnge...
1,Lamborghini_Detik,"Bak Koboi Jalanan, Pengemu...","Senin, 23 Des 2019 21:30 WIB","aksi koboi,aksi koboi peng...",Jakarta - Aksi arogan dila...,jakarta aksi arogan dilaku...
2,Lamborghini_Kompas,Geger Aksi Koboi Pengemudi...,"Kamis, 26 Desember 2019 | ...","Lamborghini koboi, Lamborg...",Kasus penodongan 2 pelajar...,kasus penodongan pelajar k...
3,Lamborghini_Kompas2,Polisi Tahan Pengembudi La...,"Rabu, 25 Desember 2019 | 0...","Mobil lamborghini, Penodon...",Polda Metro Jaya telah men...,polda metro jaya telah men...
4,Lamborghini_Merdeka,"Tembakkan Pistol ke Udara,...","Senin, 23 Desember 2019 23:06","Kriminal,Lamborghini,Penod...",Aksi koboi jalanan kembali...,aksi koboi jalanan kembali...
5,Lamborghini_Tribun,Geger Aksi Koboi Pengemudi...,"Kamis, 26 Desember 2019 23...",None,Kasus penodongan 2pelajar ...,kasus penodongan pelajar k...


##### NEWS SELECTION (BASED ON TRENDING ISSUE) 
(Dilewatkan karena tidak ada banyak berita, seleksi berita manual)

### NEWS FEATURE EXTRACTION

##### SENTENCE WEIGHTING

###### W1 based on TF ( - )

In [22]:
def tokenize_and_stem(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stemmed = stemmer.stem(text)    
    tokenized_and_stemmed = tokenize_only(stemmed)
    return tokenized_and_stemmed

In [30]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df = 0.2,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(news_df['clean'].tolist()) #fit the vectorizer to synopses

Wall time: 17 s


###### W2 sum of their TF-IDF 

## doing

In [57]:
total_num_sentences = 0
for article in (news_df['clean'].tolist()):
    total_num_sentences += len(split_into_sentences(article))

SUMMARY_LENGTH = 5
if total_num_sentences <= SUMMARY_LENGTH:
    summary = [x[1] for x in articles]

In [58]:
# build_TFIDF_model

news_tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem)
tdm = news_tfidf_vectorizer.fit_transform(news_df['isi'].tolist())  # Term-document matrix  


In [341]:
def frequency_scores(article_text):
    """ Individual (stemmed) word weights are then calculated for each
        word in the given article. Sentences are scored as the sum of their TF-IDF word frequencies.
    """

    # Add our document into the model so we can retrieve scores
    response = tfidf_vectorizer.transform([article_text])
    feature_names = tfidf_vectorizer.get_feature_names() # these are just stemmed words
    print(response)
    print(feature_names)
    word_prob = {}  # TF-IDF individual word probabilities
    for col in response.nonzero()[1]:
        word_prob[feature_names[col]] = response[0, col]
    if False: # DEBUG
        print(word_prob)

    sent_scores = []
    for sentence in article_text:
        score = 0
        sent_tokens = tokenize_and_stem(sentence)
        for token in (t for t in sent_tokens if t in word_prob):
            score += word_prob[token]

        # Normalize score by length of sentence, since we later factor in sentence length as a feature
        sent_scores.append(score / len(sent_tokens))

    return sent_scores



In [39]:
def split_into_sentences(article_text):
    return (re.split(r' *[\.\?!][\'"\)\]]* *', article_text))

D:\ITS\Mata Kuliah\Tugas Akhir\TAFinal\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arti', 'asa', 'buah', 'bum', 'enak', 'hadap', 'ikan', 'ira', 'jenak', 'kali', 'kena', 'kerja', 'khusus', 'laku', 'langsung', 'lihat', 'maksud', 'masuk', 'mena', 'nyata', 'olah', 'orang', 'rupa', 'sampa', 'sangkut', 'sekal', 'sekira', 'sin', 'tuju', 'upa', 'utama'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [37]:
article_text = (news_df['isi'].tolist())[4]

In [40]:
news_response = news_tfidf_vectorizer.transform(split_into_sentences(article_text))
news_feature_names = news_tfidf_vectorizer.get_feature_names() # these are just stemmed words

In [41]:
word_prob = {}  # TF-IDF individual word probabilities
for col in news_response.nonzero()[1]:
        word_prob[news_feature_names[col]] = news_response[0, col]
word_prob

{'koboi': 0.6585747460275427,
 'jalan': 0.4927709891621176,
 'aksi': 0.5687319721406086,
 'warna': 0.0,
 'super': 0.0,
 'sore': 0.0,
 'selatan': 0.0,
 'sabtu': 0.0,
 'oranye': 0.0,
 'orang': 0.0,
 'lamborghini': 0.0,
 'laku': 0.0,
 'kemudi': 0.0,
 'kemang': 0.0,
 'kawasan': 0.0,
 'jakarta': 0.0,
 'car': 0.0,
 'ajar': 0.0,
 'pinggir': 0.0,
 'inisial': 0.0,
 'i': 0.0,
 'canda': 0.0,
 'asyik': 0.0,
 'a': 0.0,
 'sahabat': 0.0,
 'kopi': 0.0,
 'karib': 0.0,
 'beli': 0.0,
 'mobil': 0.0,
 'lihat': 0.0,
 'kendara': 0.0,
 'impi': 0.0,
 'anak': 0.0,
 'tertawa': 0.0,
 'lirik': 0.0,
 'arah': 0.0,
 'tuh': 0.0,
 'gue': 0.0,
 'tertawa-tawa': 0.0,
 'sahut': 0.0,
 'todong': 0.0,
 'syok': 0.0,
 'pistol': 0.0,
 'kronologi': 0.0,
 'ibunda': 0.0,
 'cerita': 0.0,
 'ade': 0.0,
 'sang': 0.0,
 'milik': 0.0,
 'mewah': 0.0,
 'kaca': 0.0,
 'jendela': 0.0,
 'buka': 0.0,
 'abg': 0.0,
 'senjata': 0.0,
 'maki': 0.0,
 'langsung': 0.0,
 'teman': 0.0,
 'nodong': 0.0,
 'ngomong': 0.0,
 'lu': 0.0,
 'merdeka': 0.0,
 'hubung

In [402]:
DEBUG = False
article_scores = []
all_sent_scores = []
for article_text in news_df['isi'].tolist():
    # Add our document into the model so we can retrieve scores
    news_response = news_tfidf_vectorizer.transform(split_into_sentences(article_text))
    news_feature_names = news_tfidf_vectorizer.get_feature_names() # these are just stemmed words
    word_prob = {}  # TF-IDF individual word probabilities
    
    for col in news_response.nonzero()[1]:
        word_prob[news_feature_names[col]] = news_response[0, col]
    if DEBUG:
        print(word_prob)
    print("ini berita ke: "+ str(((news_df['isi'].tolist()).index(article_text))+1))
    sent_scores = []
    for sentence in split_into_sentences(article_text):
#         print("ini kalimat ke: "+ str(((split_into_sentences(article_text)).index(sentence))+1))
        print("Di bawah adalah perhitungan kalimat ke: "+ str(((split_into_sentences(article_text)).index(sentence))+1))
        if len(sentence) > 0:
            score = 0
            sent_tokens = tokenize_and_stem(sentence)
            for token in (t for t in sent_tokens if t in word_prob):
                score += word_prob[token]
                print("ini kata ke: "+ str(kata)+" yaitu "+token+" dengan score "+str(score))
            # Normalize score by length of sentence, since we later factor in sentence length as a feature
            sent_scores.append(score / len(sent_tokens))
            all_sent_scores.append(score / len(sent_tokens))
    article_scores.append(sent_scores)
    print("Kalimat ke: "+ str(((split_into_sentences(article_text)).index(sentence))+1)+" memiliki score "+str(score / len(sent_tokens)))

# print(article_scores)
# print(all_sent_scores)

ini berita ke: 1
Di bawah adalah perhitungan kalimat ke: 1
ini kata ke: 2 yaitu jakarta dengan score 0.4051502572994786
ini kata ke: 2 yaitu cnn dengan score 0.8615040054597964
ini kata ke: 2 yaitu indonesia dengan score 1.3178577536201144
ini kata ke: 2 yaitu mobil dengan score 1.5204328822698536
ini kata ke: 2 yaitu lamborghini dengan score 1.7230080109195929
ini kata ke: 2 yaitu laku dengan score 1.9255831395693321
ini kata ke: 2 yaitu todong dengan score 2.196319174839679
ini kata ke: 2 yaitu ajar dengan score 2.3988943034894183
ini kata ke: 2 yaitu kemang dengan score 2.6014694321391576
ini kata ke: 2 yaitu jakarta dengan score 3.006619689438636
ini kata ke: 2 yaitu selatan dengan score 3.2091948180883754
ini kata ke: 2 yaitu ganja dengan score 3.525134186973558
Di bawah adalah perhitungan kalimat ke: 2
ini kata ke: 2 yaitu kemudi dengan score 0.0
ini kata ke: 2 yaitu inisial dengan score 0.0
ini kata ke: 2 yaitu sangka dengan score 0.0
Di bawah adalah perhitungan kalimat ke: 3
in

ini berita ke: 3
Di bawah adalah perhitungan kalimat ke: 1
ini kata ke: 2 yaitu todong dengan score 0.2762249194458816
ini kata ke: 2 yaitu ajar dengan score 0.48290704263327866
ini kata ke: 2 yaitu kemang dengan score 0.6895891658206758
ini kata ke: 2 yaitu jakarta dengan score 0.8962712890080728
ini kata ke: 2 yaitu selatan dengan score 1.1029534121954698
ini kata ke: 2 yaitu orang dengan score 1.3414957251803452
ini kata ke: 2 yaitu kendara dengan score 1.5800380381652206
ini kata ke: 2 yaitu lamborghini dengan score 1.7867201613526176
ini kata ke: 2 yaitu arogansi dengan score 2.1685236052969805
ini kata ke: 2 yaitu milik dengan score 2.6456082312667313
ini kata ke: 2 yaitu mobil dengan score 2.8522903544541283
ini kata ke: 2 yaitu milik dengan score 3.329374980423879
ini kata ke: 2 yaitu senjata dengan score 3.6055998998697607
ini kata ke: 2 yaitu api dengan score 3.927944602276839
Di bawah adalah perhitungan kalimat ke: 2
Di bawah adalah perhitungan kalimat ke: 3
ini kata ke: 2 y

ini kata ke: 2 yaitu jalan dengan score 0.0
ini kata ke: 2 yaitu anak dengan score 0.0
ini kata ke: 2 yaitu mobil dengan score 0.19514905697927376
ini kata ke: 2 yaitu lamborghini dengan score 0.3902981139585475
ini kata ke: 2 yaitu kendara dengan score 0.3902981139585475
ini kata ke: 2 yaitu laku dengan score 0.5854471709378213
Di bawah adalah perhitungan kalimat ke: 8
ini kata ke: 2 yaitu mobil dengan score 0.19514905697927376
Di bawah adalah perhitungan kalimat ke: 9
ini kata ke: 2 yaitu canda dengan score 0.0
ini kata ke: 2 yaitu ajar dengan score 0.19514905697927376
ini kata ke: 2 yaitu sang dengan score 0.19514905697927376
ini kata ke: 2 yaitu milik dengan score 0.19514905697927376
ini kata ke: 2 yaitu mobil dengan score 0.3902981139585475
ini kata ke: 2 yaitu mewah dengan score 0.3902981139585475
Di bawah adalah perhitungan kalimat ke: 10
ini kata ke: 2 yaitu senjata dengan score 0.0
ini kata ke: 2 yaitu anak dengan score 0.0
Di bawah adalah perhitungan kalimat ke: 11
Kalimat ke

ini kata ke: 2 yaitu tangkap dengan score 0.0
ini kata ke: 2 yaitu milik dengan score 0.5109461821707708
ini kata ke: 2 yaitu lamborghini dengan score 0.5109461821707708
ini kata ke: 2 yaitu inisial dengan score 0.5109461821707708
ini kata ke: 2 yaitu aksi dengan score 0.5109461821707708
ini kata ke: 2 yaitu bak dengan score 0.5109461821707708
ini kata ke: 2 yaitu koboi dengan score 0.5109461821707708
ini kata ke: 2 yaitu jalan dengan score 0.5109461821707708
ini kata ke: 2 yaitu kemang dengan score 0.7322977747753189
ini kata ke: 2 yaitu polisi dengan score 0.7322977747753189
ini kata ke: 2 yaitu laku dengan score 0.7322977747753189
ini kata ke: 2 yaitu geledah dengan score 0.7322977747753189
ini kata ke: 2 yaitu rumah dengan score 0.7322977747753189
ini kata ke: 2 yaitu sita dengan score 0.7322977747753189
Di bawah adalah perhitungan kalimat ke: 5
ini kata ke: 2 yaitu polisi dengan score 0.0
ini kata ke: 2 yaitu polres dengan score 0.0
ini kata ke: 2 yaitu metro dengan score 0.0
ini 

In [389]:
print(article_scores[4])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [355]:
(news_df['isi'].tolist())[4]

'Merdeka.com - Aksi koboi jalanan kembali terjadi. Pelaku seorang pengemudi super car Lamborghini berwarna oranye kepada seorang pelajar di bawah umum di kawasan Kemang, Jakarta Selatan, Sabtu (21/12) sore lalu. Pelajar inisial A dan I awalnya tengah asyik bercanda di pinggir jalan Kemang Selatan, Jakarta. Kedua sahabat karib ini hendak membeli kopi. Saat di perjalanan, keduanya anak ini melihat mobil impian mereka, Lamborghini yang dikendarai pelaku. Keduanya pun tertawa sambil melirik ke arah mobil itu.  "Mobil gue tuh," kata A. "Itu mobil gue," sahut I sambil tertawa-tawa. Kronologi itu diceritakan oleh ibunda, A, Ade yang syok anaknya ditodong pistol. Di tengah candaan kedua anak ABG itu, sang pemilik mobil mewah tiba-tiba membuka kaca jendela. Pelaku langsung mengeluarkan senjata sekaligus makian kepada anak-anak tersebut. "Dia langsung menodongkan pistol ke arah temannya A, dia nodong ngomong apa lu ****?" ucap Ade yang diceritakan langsung oleh anaknya saat dihubungi merdeka.com

In [ ]:
# Add our document into the model so we can retrieve scores
news_response = news_tfidf_vectorizer.transform(split_into_sentences(article_text))
news_feature_names = news_tfidf_vectorizer.get_feature_names() # these are just stemmed words
word_prob = {}  # TF-IDF individual word probabilities

In [347]:
article_text = (news_df['isi'].tolist())[0]
article_text

'Jakarta, CNN Indonesia -- Pngendara mobil Lamborghini yang melakukan penodongan kepada dua pelajar di Kemang, Jakarta Selatan positif menggunakan narkotika jenis ganja. Pengemudi berinisial AM itu saat ini sudah ditetapkan sebagai tersangka. "Tim reserse Polres Jakarta Selatan coba mendalami kemungkinan pelaku mabok atau tidak. Ternyata positif ganja," kata Kepala Bidang Humas Polda Metro Jaya Kombes Yusri Yunus di Polres Metro Jakarta Selatan, Kebayoran Lama, Jakarta Selatan pada Selasa (24/12). Kendati demikian, tidak ditemukan barang bukti narkotika di mobil maupun rumah AM. Namun polisi terus mendalami kemungkinan penyalahgunaan narkotika oleh AM ini. Polisi menyebut AM merupakan seorang pengusaha properti. Ia juga merupakan pemilik Lamborghini jenis Gallardo tersebut. Atas perbuatannya ia dijerat dengan Pasal 335 KUHP tentang perbuatan tidak menyenangkan dengan ancaman pidana 1 tahun penjara. Peristiwa penodongan oleh pengemudi Lamborghini terhadap dua pelajar SMA asal Jakarta it

In [45]:
word_prob={}
for col in news_response.nonzero()[1]:
    word_prob[news_feature_names[col]] = news_response[1, col]
if DEBUG:
    print(word_prob)
word_prob

NameError: name 'DEBUG' is not defined

In [51]:
# used
article_text = (news_df['isi'].tolist())[3]
# Add our document into the model so we can retrieve scores
news_response = news_tfidf_vectorizer.transform(split_into_sentences(article_text))
news_feature_names = news_tfidf_vectorizer.get_feature_names() # these are just stemmed words

each_news_df = pd.DataFrame(columns=['sentence','freq_score'])

sent_scores = []
for sentence in split_into_sentences(article_text):
    word_prob = {}  # TF-IDF individual word probabilities
    article_scores = []
    for col in news_response.nonzero()[1]:
        word_prob[news_feature_names[col]] = news_response[0, col]
    if False:
        print(word_prob)
    print("ini berita ke: "+ str(((news_df['isi'].tolist()).index(article_text))+1))

    print("Di bawah adalah perhitungan kalimat ke: "+ str(((split_into_sentences(article_text)).index(sentence))+1))
    if len(sentence) > 0:
        score = 0
        sent_tokens = tokenize_and_stem(sentence)
        kata = 0
        for token in (t for t in sent_tokens if t in word_prob):
            kata+=1
            score += word_prob[token]
            print("ini kata ke: "+ str(kata)+" yaitu "+token+" dengan score "+str(score))
        # Normalize score by length of sentence, since we later factor in sentence length as a feature
        sent_scores.append(score / len(sent_tokens))
    print("Kalimat ke: "+ str(((split_into_sentences(article_text)).index(sentence))+1)+" memiliki score "+str(score / len(sent_tokens)))
    each_news_df = each_news_df.append({'sentence':sentence, 'freq_score':(score / len(sent_tokens))}, ignore_index=True)
    print(each_news_df)
# print(article_scores)

ini berita ke: 4
Di bawah adalah perhitungan kalimat ke: 1
ini kata ke: 1 yaitu polda dengan score 0.32487187322229916
ini kata ke: 2 yaitu metro dengan score 0.5007351599118666
ini kata ke: 3 yaitu jaya dengan score 0.8256070331341657
ini kata ke: 4 yaitu tahan dengan score 1.2217853329063075
ini kata ke: 5 yaitu kemudi dengan score 1.4247580647957276
ini kata ke: 6 yaitu mobil dengan score 1.600621351485295
ini kata ke: 7 yaitu lamborghini dengan score 1.7764846381748622
ini kata ke: 8 yaitu laku dengan score 1.9523479248644295
ini kata ke: 9 yaitu aksi dengan score 2.1553206567538497
ini kata ke: 10 yaitu pistol dengan score 2.390357056855413
ini kata ke: 11 yaitu bak dengan score 2.6646362152946756
ini kata ke: 12 yaitu orang dengan score 2.8676089471840958
ini kata ke: 13 yaitu koboi dengan score 3.102645347285659
ini kata ke: 14 yaitu hadap dengan score 3.3769245057249218
ini kata ke: 15 yaitu ajar dengan score 3.552787792414489
ini kata ke: 16 yaitu kawasan dengan score 3.728651

ini kata ke: 1 yaitu jalan dengan score 0.0
ini kata ke: 2 yaitu anak dengan score 0.0
ini kata ke: 3 yaitu lihat dengan score 0.0
ini kata ke: 4 yaitu mobil dengan score 0.17586328668956736
ini kata ke: 5 yaitu lamborghini dengan score 0.3517265733791347
ini kata ke: 6 yaitu kendara dengan score 0.3517265733791347
ini kata ke: 7 yaitu laku dengan score 0.5275898600687021
Kalimat ke: 7 memiliki score 0.043965821672391846
                                                                                                                                                                                          sentence  \
0                            Polda Metro Jaya telah menahan pengemudi mobil lamborghini yang melakukan aksi menggunakan pistol bak seorang "koboi" terhadap dua pelajar di kawasan Jakarta Selatan   
1                                                                                     Selain menahan pengemudi, Polda Metro Jaya juga menunjukkan senjata api yang digunakan pengem

In [52]:
each_news_df

,sentence,freq_score
0,"Polda Metro Jaya telah menahan pengemudi mobil lamborghini yang melakukan aksi menggunakan pistol bak seorang ""koboi"" terhadap dua pelajar di kawasan Jakarta Selatan",0.177408
1,"Selain menahan pengemudi, Polda Metro Jaya juga menunjukkan senjata api yang digunakan pengemudi lamborghini",0.128828
2,Kabid Humas Polda Metro Jaya Kombes Yusri Yunus menyatakan tindakan pengemudi lamborgini melepaskan tembakan ke atas dengan pistol sebanyak tiga kali karena tidak terima dengan ucapan pelajar,0.053314
3,"Aksi koboi pengemudi lamborghini terjadi di kawasan Kemang, Jakarta Selatan",0.134444
4,"Aksi koboi jalanan bermula saat pelajar berinisial A dan I tengah asyik bercanda di pinggir Jalan Kemang Selatan, Jakarta pada Sabtu sore 21 Desember 2019",0.041983
5,Keduanya hendak membeli kopi,0.000000
6,"Saat di perjalanan, keduanya anak ini melihat mobil Lamborghini yang dikendarai pelaku",0.043966
7,Keduanya pun tertawa sambil melirik ke arah mobil itu,0.019540
8,"Di tengah candaan kedua pelajar itu, sang pemilik mobil mewah tiba-tiba membuka kaca jendela",0.025123
9,Dia langsung mengeluarkan senjata sekaligus makian kepada anak-anak tersebut,0.000000


In [49]:
#1 
sorted_each_news_df = each_news_df.sort_values(by=['score'], ascending=False)
pd.set_option('display.max_colwidth', 300)
sorted_each_news_df

,sentence,score
0,"Polda Metro Jaya telah menahan pengemudi mobil lamborghini yang melakukan aksi menggunakan pistol bak seorang ""koboi"" terhadap dua pelajar di kawasan Jakarta Selatan",0.177408
3,"Aksi koboi pengemudi lamborghini terjadi di kawasan Kemang, Jakarta Selatan",0.134444
1,"Selain menahan pengemudi, Polda Metro Jaya juga menunjukkan senjata api yang digunakan pengemudi lamborghini",0.128828
2,Kabid Humas Polda Metro Jaya Kombes Yusri Yunus menyatakan tindakan pengemudi lamborgini melepaskan tembakan ke atas dengan pistol sebanyak tiga kali karena tidak terima dengan ucapan pelajar,0.053314
6,"Saat di perjalanan, keduanya anak ini melihat mobil Lamborghini yang dikendarai pelaku",0.043966
4,"Aksi koboi jalanan bermula saat pelajar berinisial A dan I tengah asyik bercanda di pinggir Jalan Kemang Selatan, Jakarta pada Sabtu sore 21 Desember 2019",0.041983
8,"Di tengah candaan kedua pelajar itu, sang pemilik mobil mewah tiba-tiba membuka kaca jendela",0.025123
7,Keduanya pun tertawa sambil melirik ke arah mobil itu,0.019540
5,Keduanya hendak membeli kopi,0.000000
9,Dia langsung mengeluarkan senjata sekaligus makian kepada anak-anak tersebut,0.000000


In [462]:
sorted_each_news_df['sentence'].tolist()

['Pelaku seorang pengemudi super car Lamborghini berwarna oranye kepada seorang pelajar di bawah umum di kawasan Kemang, Jakarta Selatan, Sabtu (21/12) sore lalu',
 'Pelajar inisial A dan I awalnya tengah asyik bercanda di pinggir jalan Kemang Selatan, Jakarta',
 'Sementara itu, Kasat Reskrim Polres Metro Jakarta Selatan Kompol Andi Sinjaya mengakui telah menerima laporan tentang pengemudi Lamborghini koboi tersebut',
 'Ade pun telah melaporkan hal ini ke Polres Jakarta Selatan',
 'Seingat, A setidaknya lebih dari tiga kali orang tersebut menembakkan pistol ke atas',
 'Pelaku sambil menodongkan pistolnya sambil berkendara',
 'A pun menjadi bulan-bulanan makian pelaku',
 'Namun teriakan Aiman dianggap perlawanan oleh pelaku',
 'Saat di perjalanan, keduanya anak ini melihat mobil impian mereka, Lamborghini yang dikendarai pelaku',
 'A pun sempat berteriak karena suara bising mobil, dia pun merasa tidak melakukan kesalahan kepada pelaku',
 'Si pelaku akhirnya hendak keluar dari mobil mewa

In [459]:
#         word_prob[news_feature_names[col]] = news_response[index_kalimat, col]
sorted_each_news_df['sentence'].tolist()

['Aksi koboi jalanan kembali terjadi',
 'Pelaku sambil menodongkan pistolnya sambil berkendara',
 '"Mobil gue tuh," kata A',
 'Tapi anak saya tetap jalan," terang Ade',
 'Meminta si pemilik mobil segera jalan, karena memicu kemacetan',
 'Si pelaku akhirnya hendak keluar dari mobil mewahnya tersebut',
 'Namun teriakan Aiman dianggap perlawanan oleh pelaku',
 'Pelaku langsung mengeluarkan senjata sekaligus makian kepada anak-anak tersebut',
 'Pelajar inisial A dan I awalnya tengah asyik bercanda di pinggir jalan Kemang Selatan, Jakarta',
 'Ade pun telah melaporkan hal ini ke Polres Jakarta Selatan',
 'Polisi pun telah memeriksanya untuk mengusut kasus tersebut',
 'Saat di perjalanan, keduanya anak ini melihat mobil impian mereka, Lamborghini yang dikendarai pelaku',
 'A pun menjadi bulan-bulanan makian pelaku',
 'Pelaku seorang pengemudi super car Lamborghini berwarna oranye kepada seorang pelajar di bawah umum di kawasan Kemang, Jakarta Selatan, Sabtu (21/12) sore lalu',
 'Di tengah can

In [411]:
split_into_sentences((news_df['isi'].tolist())[4])[18]
# (news_df['isi'].tolist())[1]

'A pun menjadi bulan-bulanan makian pelaku'

In [441]:
sent_scores = sent_scores1

In [ ]:
pd.DataFrame(columns=['words','frequency'])

In [160]:
# def frequency_scores(article_text):
""" Individual (stemmed) word weights are then calculated for each
    word in the given article. Sentences are scored as the sum of their TF-IDF word frequencies.
"""

# Add our document into the model so we can retrieve scores
response = tfidf_vectorizer.transform([sample_article])
feature_names = tfidf_vectorizer.get_feature_names() # these are just stemmed words
print(response)
print(feature_names)
word_prob = {}  # TF-IDF individual word probabilities
for col in response.nonzero()[1]:
    word_prob[feature_names[col]] = response[0, col]
if False: # DEBUG
    print(word_prob)

sent_scores = []
for sentence in sample_article:
    score = 0
    sent_tokens = tokenize_and_stem(sentence)
    for token in (t for t in sent_tokens if t in word_prob):
        score += word_prob[token]

    # Normalize score by length of sentence, since we later factor in sentence length as a feature
    sent_scores.append(score / len(sent_tokens))

# return sent_scores



  (0, 596)	0.07743209993333898
  (0, 595)	0.07743209993333898
  (0, 594)	0.07743209993333898
  (0, 589)	0.07743209993333898
  (0, 581)	0.07743209993333898
  (0, 571)	0.07743209993333898
  (0, 568)	0.16806036644652794
  (0, 567)	0.07743209993333898
  (0, 566)	0.3097283997333559
  (0, 560)	0.07743209993333898
  (0, 559)	0.07743209993333898
  (0, 558)	0.07743209993333898
  (0, 555)	0.06537349938992297
  (0, 554)	0.06537349938992297
  (0, 553)	0.06537349938992297
  (0, 545)	0.06537349938992297
  (0, 543)	0.07743209993333898
  (0, 537)	0.056020122148842645
  (0, 523)	0.06537349938992297
  (0, 519)	0.06537349938992297
  (0, 501)	0.07743209993333898
  (0, 490)	0.07743209993333898
  (0, 480)	0.07743209993333898
  (0, 479)	0.2801006107442132
  (0, 478)	0.07743209993333898
  :	:
  (0, 162)	0.07743209993333898
  (0, 161)	0.07743209993333898
  (0, 160)	0.06537349938992297
  (0, 155)	0.056020122148842645
  (0, 148)	0.13074699877984594
  (0, 132)	0.11204024429768529
  (0, 131)	0.2801006107442132
  (

ZeroDivisionError: division by zero

In [161]:
sent_scores

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [156]:
FS = frequency_scores(sample_article)

  (0, 596)	0.07743209993333898
  (0, 595)	0.07743209993333898
  (0, 594)	0.07743209993333898
  (0, 589)	0.07743209993333898
  (0, 581)	0.07743209993333898
  (0, 571)	0.07743209993333898
  (0, 568)	0.16806036644652794
  (0, 567)	0.07743209993333898
  (0, 566)	0.3097283997333559
  (0, 560)	0.07743209993333898
  (0, 559)	0.07743209993333898
  (0, 558)	0.07743209993333898
  (0, 555)	0.06537349938992297
  (0, 554)	0.06537349938992297
  (0, 553)	0.06537349938992297
  (0, 545)	0.06537349938992297
  (0, 543)	0.07743209993333898
  (0, 537)	0.056020122148842645
  (0, 523)	0.06537349938992297
  (0, 519)	0.06537349938992297
  (0, 501)	0.07743209993333898
  (0, 490)	0.07743209993333898
  (0, 480)	0.07743209993333898
  (0, 479)	0.2801006107442132
  (0, 478)	0.07743209993333898
  :	:
  (0, 162)	0.07743209993333898
  (0, 161)	0.07743209993333898
  (0, 160)	0.06537349938992297
  (0, 155)	0.056020122148842645
  (0, 148)	0.13074699877984594
  (0, 132)	0.11204024429768529
  (0, 131)	0.2801006107442132
  (

In [159]:
FS

###### W3 position in the article 

In [176]:
def position_score(i, size):
        """ Yields a value between (0,1), corresponding to sentence's position in the article.
            Assuming that sentences at the very beginning and ends of the article have a higher weight. 
            Values borrowed from https://github.com/xiaoxu193/PyTeaser, modified by Reinardus (change 1st and 2nd)
        """
        relative_position = i / size
        if 0 < relative_position <= 0.1:
            return 0.23
        elif 0.1 < relative_position <= 0.2:
            return 0.17
        elif 0.2 < relative_position <= 0.3:
            return 0.14
        elif 0.3 < relative_position <= 0.4:
            return 0.08
        elif 0.4 < relative_position <= 0.5:
            return 0.05
        elif 0.5 < relative_position <= 0.6:
            return 0.04
        elif 0.6 < relative_position <= 0.7:
            return 0.06
        elif 0.7 < relative_position <= 0.8:
            return 0.04
        elif 0.8 < relative_position <= 0.9:
            return 0.04
        elif 0.9 < relative_position <= 1.0:
            return 0.15
        else:
            return 0

In [218]:
position_score_each_news = []

for news in (news_df['isi'].tolist()):
    news_score = []
    index = 1
    sentences = re.split(r' *[\.\?!][\'"\)\]]* *', news)
#     print("Berita ke: "+ str(index)+ " terdiri dari "+ str(len(sentences)) + " kalimat")
    for sentence in sentences:
        news_score.append(position_score(index,len(sentences)))
        index+=1
    position_score_each_news.append(news_score)

# position_score_each_news 

###### W4 words common to the headline (unfinished)

In [115]:
def headline_score(self, headline, sentence):
    """ Gives sentence a score between (0,1) based on percentage of words common to the headline. """
    title_stems = [stemmer.stem(w) for w in headline if w not in stop_words]
    sentence_stems = [stemmer.stem(w) for w in sentence if w not in stop_words]
    count = 0.0
    for word in sentence_stems:
        if word in title_stems:
            count += 1.0
    score = count / len(title_stems)
    return score

###### W5 words common to the trending issue (unfinished)


In [118]:
def trending_issue_score(self, trending_issue, sentence):
    """ Gives sentence a score between (0,1) based on percentage of words common to the trending issue. """
    stemmer = factory.create_stemmer()
    sentence_stems = [stemmer.stem(w) for w in sentence if w not in stop_words]
    count = 0.0
    for word in sentence_stems:
        if word in trending_issue:
            count += 1.0
    score = count / len(trending_issue)
    return score

###### W6 named entity contained (unfinished)

In [112]:
def entity_score(self, sentence):
    """ Gives sentence a score 1 for every named entity contained. """
    result = AnagoModel.analyze(sentence.split())
    score = len(result.get("entities"))
    return score

###### W7 Sentence Length (unfinished)

In [114]:
def length_score(self, sentence):
    ideal_sent_length = 20.0
    """ Gives sentence score between (0,1) based on how close sentence's length is to the ideal length."""
    len_diff = math.fabs(ideal_sent_length - len(sentence))
    return len_diff / ideal_sent_length


In [ ]:
legth_score_each_news = []

for news in (news_df['isi'].tolist()):
    news_score = []
    index = 1
    sentences = re.split(r' *[\.\?!][\'"\)\]]* *', news)
#     print("Berita ke: "+ str(index)+ " terdiri dari "+ str(len(sentences)) + " kalimat")
    for sentence in sentences:
        news_score.append(position_score(index,len(sentences)))
        index+=1
    position_score_each_news.append(news_score)


In [ ]:
for i in range()

##### W7 multidocument weight (Cosine Simmilarity) (unfinished)

## doing

###### WTot (unfinished)

In [ ]:
""" Assigns each sentence in the document a score based on the sum of features values.
    Based on 4 features: relevance to headline, length, sentence position, and TF*IDF frequency.
"""

headline = article[0]
sentences = self.split_into_sentences(article[1])
frequency_scores = self.frequency_scores(article[1])

xxx total_score_each_news = []

for i, s in enumerate(sentences):
    headline_score = self.headline_score(headline, s) * 1.5
    length_score = self.length_score(self.split_into_words(s)) * 1.0
    position_score = self.position_score(float(i+1), len(sentences)) * 1.0
    frequency_score = frequency_scores[i] * 4
    # enitas bernama
    score = (headline_score + frequency_score + length_score + position_score) / 4.0
    self._scores[s] = score

In [ ]:
each_news_weight = []
all_sentence_weight = []
for index, row in news_df.iterrows():
    news_weight = []
    for sentence in row['clean']:
        sentence_score = 0
        
        sentence_score += 
        news_weight.append(sentence_score)
        all_sentence_weight.append(sentence_score)
    each_news_weight.append(news_weight)

In [ ]:

scores = Counter()
for article in articles:
    self.score(article)

highest_scoring = scores.most_common(SUMMARY_LENGTH)
if DEBUG:
    print(highest_scoring)

print("## Headlines: ")
for article in self._articles:
    print("- " + article[0])

# Appends highest scoring "representative" sentences, returns as a single summary paragraph.
return ' '.join([sent[0] for sent in highest_scoring])

### GENERATING SUMMARY

In [ ]:
# mengurutkan bobot kalimat berita

# menentukan jumlah kalimat hasil ringkasan
def generate_summaries(self):
    """ If article is shorter than the desired summary, just return the original articles."""

    # Rare edge case (when total num sentences across all articles is smaller than desired summary length)
    total_num_sentences = 0
    for article in self._articles:
        total_num_sentences += len(self.split_into_sentences(article[1]))

    if total_num_sentences <= SUMMARY_LENGTH:
        return [x[1] for x in self._articles]

    self.build_TFIDF_model()  # only needs to be done once

    self._scores = Counter()
    for article in self._articles:
        self.score(article)

    highest_scoring = self._scores.most_common(SUMMARY_LENGTH)
    if DEBUG:
        print(highest_scoring)

    print("## Headlines: ")
    for article in self._articles:
        print("- " + article[0])

    # Appends highest scoring "representative" sentences, returns as a single summary paragraph.
    return ' '.join([sent[0] for sent in highest_scoring])


In [ ]:
for i in range(N_CLUSTER):
    cluster_tf, cluster_terms =
    get_tf(non_trivial[cluster_id == i].tolist(),get_terms = True)
    cluster_tfidf = get_tfidf(non_trivial[cluster_id == i].tolist(),normalize = True)
    cluster_wf = np.array(get_wf(non_trivial[cluster_id == i].tolist()).values())
    selected = np.argwhere(np.logical_and(np.logical_and ((cluster_tfidf.max(axis = 0) > TFIDF), cluster_wf > WF), cluster_tf.max(axis = 0) > TF)).T[0]
    keywords.extend([cluster_terms[idx] for idx in selected])self.result_dict['result'][i]['selected_terms']= [cluster_terms[idx] for idx in selected]
    if len(self.result_dict['result'][i]['selected_terms']) > t_max:
        t_max = len(self.result_dict['result'][i]['selected_terms'])
        best_cluster = i
    self.result_dict['result'][i]['members'] = cluster_tf.shape[0]
self.result_dict['result'][i]['cluster_terms'] = cluster_terms
max_score = 0
for i in range(N_CLUSTER):
    term_clust = []
    for trm in non_trivial[cluster_id == i]:
        term_clust.extend(trm)
    score = 0
    for keyword in keywords:
        cou = term_clust.count(keyword)
        if cou > CI_THRES:
            score += np.log(term_clust.count(keyword) + 1)
        if max_score < score:
            max_score = score
            best_cluster = i
            self.result_dict['result'][i]['score'] = core
self.selected_terms_final = self.result_dict['result'][best_cluster]['select ed_terms']                                                                

### TESTING

In [ ]:
precision

In [ ]:
for sentence in summary:
    

### FLASK AS FRONT END

##### jangan dioprek begin

In [445]:
# Clustering menggunakan K-Means dengan fitur tf-idf
## jangan di apa-apain dulu, memperlihatkan ekstraksi isu yang nyaris berhasil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# documents = indonesia['clean'].tolist()
# vectorizer = TfidfVectorizer(stop_words=stopwords)
# X = vectorizer.fit_transform(documents)

true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(tfidf_matrix)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=5, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [215]:
## Most representative words
## jangan di apa-apain dulu, memperlihatkan ekstraksi isu yang nyaris berhasil
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 garuda
 indonesia
 ciduk
 smoldotid
 pria
 kerja
 dok
 situ
 sampe
 diskon
Cluster 1:
 camat
 bah
 polri
 mabes
 kadisono
 desa
 dusun
 kabupaten
 tegaltirto
 antiteror
Cluster 2:
 tangkap
 teroris
 duga
 densus
 sleman
 bah
 vivanews
 dirgahayurith
 antiteror
 diy
Cluster 3:
 aman
 teroris
 yogya
 duga
 kota
 orang
 densus
 sleman
 kenal
 warga
Cluster 4:
 geledah
 rumah
 densus
 sleman
 taucepattanpabatas
 beritaterkini
 tangkap
 orang
 milik
 antiteror


##### jangan dioprek end